# SPOR NW 2025: Exploring the Potential of Electronic Medical Records

Based off the GitHub Repo at:
https://github.com/centre-for-health-informatics/SPORNW-2025-EMR-Workshop

## Load some made up discharge summaries

In [1]:
import pandas as pd

In [ ]:
# A little code to wrap long lines of text in colab
from IPython.display import HTML, display
def set_css():
    display(HTML('''
      <style>
        pre {
            white-space: pre-wrap;
        }
      </style>
      '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
dsum = pd.read_csv("https://raw.githubusercontent.com/centre-for-health-informatics/SPORNW-2025-EMR-Workshop/refs/heads/master/synthetic_discharge_summaries.csv")
print(dsum.shape)
dsum.head(2)

(20, 4)


,title,summary,hypertension-status,Age
0,Pneumonia (Community-Acquired),The patient is a 67-year-old female with a his...,1,67
1,Acute Cholecystitis,The patient is a 49-year-old male with a BMI o...,0,49


# Data Overview

In [3]:
dsum['hypertension-status'].value_counts()

hypertension-status
0    14
1     6
Name: count, dtype: int64

In [4]:
(dsum['hypertension-status']==1).sum()/dsum.shape[0]

0.3

In [5]:
dsum['Age'].describe()

count    20.000000
mean     47.350000
std      16.840662
min      16.000000
25%      33.250000
50%      50.500000
75%      59.000000
max      72.000000
Name: Age, dtype: float64

In [6]:
dsum['summary'].apply(len).describe()

count      20.000000
mean      724.650000
std       210.718389
min       504.000000
25%       596.000000
50%       665.500000
75%       763.250000
max      1361.000000
Name: summary, dtype: float64

In [7]:
print(dsum['summary'].iloc[0])

The patient is a 67-year-old female with a history of hypertension and type 2 diabetes mellitus. She presented with a 3-day history of fever, productive cough, and shortness of breath. Her physical exam revealed crackles in the right lower lung field. Family history includes a mother with a history of lung cancer and a father who had hypertension and coronary artery disease.

The patient was diagnosed with community-acquired pneumonia (CAP) and treated with a course of intravenous ceftriaxone and azithromycin. Her fever resolved by day three, and she demonstrated significant clinical improvement. The patient was switched to oral antibiotics on day five and discharged after a 7-day course.

She was advised to monitor her blood glucose levels closely and avoid smoking, as it may worsen her lung condition. The patient was instructed to follow up with her primary care physician within one week to ensure continued improvement.


# Determining disease status with Natural Language Processing (NLP)
Let's try and determine hypertension status from these discharge summaries

## Keyword Search

In [8]:
htn_1 = dsum['summary'].str.contains('hypertension')
print(htn_1.sum())
htn_1.head()

7


0     True
1    False
2    False
3    False
4     True
Name: summary, dtype: bool

## How well does this work?
 - what metrics can we use to assess the performance of our methods

### Accuracy
- What fraction do we get correct, where 1 is perfect and 0 is perfectly wrong

In [9]:
def acc(reference_labels, our_predictions):
    ll = reference_labels == our_predictions
    return ll.sum()/ll.shape[0]

acc(dsum['hypertension-status'],htn_1)

0.65

### PPV

In [10]:
def ppv(reference_labels, our_predictions):
    ll = (reference_labels == 1) & (our_predictions == 1)
    return ll.sum()/our_predictions.sum()

ppv(dsum['hypertension-status'],htn_1)

0.42857142857142855

### Sensitivity

In [11]:
def sen(reference_labels, our_predictions):
    ll = (reference_labels == 1) & (our_predictions == 1)
    return ll.sum()/reference_labels.sum()

sen(dsum['hypertension-status'],htn_1)

0.5

### NPV

In [12]:
def npv(reference_labels, our_predictions):
    ll = (reference_labels == 0) & (our_predictions == 0)
    return ll.sum()/(~our_predictions).sum()

npv(dsum['hypertension-status'],htn_1)

0.7692307692307693

### Specificity

In [13]:
def spe(reference_labels, our_predictions):
    ll = (reference_labels == 0) & (our_predictions == 0)
    return ll.sum()/(reference_labels==0).sum()

spe(dsum['hypertension-status'],htn_1)

0.7142857142857143

### Put them all together in one performance function

In [14]:
def stats(reference_labels, our_predictions):
    print(
        f"accuracy = {acc(reference_labels, our_predictions):.2}",
        f"PPV = {ppv(reference_labels, our_predictions):.2}",
        f"Sensitivity = {sen(reference_labels, our_predictions):.2}",
        f"NPV = {npv(reference_labels, our_predictions):.2}",
        f"Specificity = {spe(reference_labels, our_predictions):.2}",
        sep='\n',
    )
stats(dsum['hypertension-status'],htn_1)

accuracy = 0.65
PPV = 0.43
Sensitivity = 0.5
NPV = 0.77
Specificity = 0.71


## Where are we getting it wrong?

### False Positives

In [15]:
def fp(reference_labels, our_predictions):
    ll = (reference_labels == 0) & (our_predictions == 1)
    n = ll.sum()
    print("number: ", n)
    if n > 0:
        print("index: ", dsum.loc[ll].index[0])
        print("summary:\n",dsum['summary'].loc[ll].iloc[0])
    
fp(dsum['hypertension-status'],htn_1)

number:  4
index:  4
summary:
 The patient is a 58-year-old diabetic male presenting with redness, warmth, and swelling over his left lower leg. No history of hypertension. He denied any trauma or recent infection in the area. Physical exam revealed localized cellulitis without signs of abscess formation. His blood glucose levels were elevated at the time of admission, reflecting poor diabetes control.

The patient was treated with IV antibiotics (vancomycin and piperacillin-tazobactam) for 48 hours, followed by a 7-day course of oral antibiotics (cephalexin). Blood glucose levels were closely monitored, and adjustments were made to his diabetic regimen during hospitalization.

The patient was discharged with instructions to keep the leg elevated, monitor for signs of worsening infection, and follow up with his primary care physician in one week.


### False Negatives

In [16]:
def fn(reference_labels, our_predictions):
    ll = (reference_labels == 1) & (our_predictions == 0)
    n = ll.sum()
    print("number: ", n)
    if n > 0:
        print("index: ", dsum.loc[ll].index[0])
        print("summary:\n",dsum['summary'].loc[ll].iloc[0])
    
fn(dsum['hypertension-status'],htn_1)

number:  3
index:  2
summary:
 The patient is a 62-year-old male with a history of gout. Hypertension present for the past 5 years. He presented with severe pain and swelling in his right great toe. His uric acid level was elevated, and joint aspiration confirmed monosodium urate crystals.

The patient was treated with colchicine and NSAIDs for symptom management, and his uric acid levels were adjusted with allopurinol. He was discharged after 48 hours of treatment, with instructions to avoid purine-rich foods and alcohol.

He was advised to follow up with his rheumatologist in one month for further management.


## Regular Expressions
https://www.rexegg.com/regex-quickstart.php

### Can We clear up our False Negatives first?

In [17]:
htn_2 = dsum['summary'].str.contains("(?i)hypertension",regex=True)
fn(dsum['hypertension-status'],htn_2)

number:  2
index:  3
summary:
 The patient is a 55-year-old male with intermittent asthma. He presented with wheezing, shortness of breath, and cough, which had worsened over the past 48 hours following a viral upper respiratory tract infection. History of HTN. Family history includes a mother with asthma and a brother with allergic rhinitis.

The patient was treated with nebulized albuterol and systemic corticosteroids. He showed rapid improvement and was discharged after 24 hours with instructions to use a prescribed inhaler (albuterol) and a 5-day course of prednisone.

The patient was advised to avoid triggers such as smoke and dust and to follow up with his pediatric pulmonologist within one week.


In [18]:
htn_3 = dsum['summary'].str.contains("(?i)(?:hypertension|htn)",regex=True)
fn(dsum['hypertension-status'],htn_3)

number:  1
index:  6
summary:
 The patient is a 58-year-old female  presenting with severe headache, blurred vision, and chest discomfort, suggesting a hypertensive crisis. Her blood pressure was recorded at 220/130 mmHg upon arrival.

The patient was started on IV labetalol and was closely monitored in the ICU. Her blood pressure was controlled over 48 hours, and she was transitioned to oral antihypertensives. Upon discharge, she was instructed to monitor her blood pressure at home and follow up with her cardiologist in one week.


In [19]:
htn_4 = dsum['summary'].str.contains("(?i)(?:hypertension|htn|hypertensive)",regex=True)
fn(dsum['hypertension-status'],htn_4)

number:  0


### How do the False Positives look now after those changes?

In [20]:
fp(dsum['hypertension-status'],htn_4)

number:  5
index:  1
summary:
 The patient is a 49-year-old male with a BMI of 35 and a history of gallstones, and struck by lightning 5 years ago. He presented with a 48-hour history of severe right upper quadrant pain, nausea, and vomiting. Ultrasound confirmed acute cholecystitis with no signs of gallbladder perforation or abscess formation. The patient has a known family history of gallbladder disease in his mother, who underwent cholecystectomy. His father had type 2 diabetes, and he is currently overweight with elevated blood sugar.

The patient underwent a laparoscopic cholecystectomy within 24 hours of presentation. Intraoperative findings included an inflamed but intact gallbladder with no complications. His post-operative recovery was uncomplicated, and he was started on clear liquids within 6 hours.

Discharge instructions included a low-fat diet for the next few weeks and recommendations for weight management. He is prescribed a 7-day course of oral antibiotics (ciprofloxac

#### Can use regex word boundaries to make sure we're searching for individual words/acronyms

In [21]:
htn_5 = dsum['summary'].str.contains(r"(?i)\b(?:hypertension|htn|hypertensive)\b",regex=True)
fp(dsum['hypertension-status'],htn_5)

number:  4
index:  4
summary:
 The patient is a 58-year-old diabetic male presenting with redness, warmth, and swelling over his left lower leg. No history of hypertension. He denied any trauma or recent infection in the area. Physical exam revealed localized cellulitis without signs of abscess formation. His blood glucose levels were elevated at the time of admission, reflecting poor diabetes control.

The patient was treated with IV antibiotics (vancomycin and piperacillin-tazobactam) for 48 hours, followed by a 7-day course of oral antibiotics (cephalexin). Blood glucose levels were closely monitored, and adjustments were made to his diabetic regimen during hospitalization.

The patient was discharged with instructions to keep the leg elevated, monitor for signs of worsening infection, and follow up with his primary care physician in one week.


### How are we doing now

In [22]:
stats(dsum['hypertension-status'],htn_5)

accuracy = 0.8
PPV = 0.6
Sensitivity = 1.0
NPV = 1.0
Specificity = 0.71


## Negations
Popular **NegEx** algorithm is a much more sophisticated version of what I'll show here
Chapman WW, Bridewell W, Hanbury P, Cooper GF, Buchanan BG. A Simple Algorithm for Identifying Negated Findings and Diseases in Discharge Summaries. Journal of Biomedical Informatics. 2001;34(5):301-310. doi:10.1006/jbin.2001.1029


In [23]:
def get_htn_6(dsum):
    la = dsum.str.contains(r"(?i)\b(?:hypertension|htn|hypertensive)\b",regex=True)
    ln = dsum.str.contains(r"(?i)\bno\b.*\b(?:hypertension|htn|hypertensive)\b",regex=True)
    return la & ~ln

htn_6 = get_htn_6(dsum['summary'])
fp(dsum['hypertension-status'],htn_6)

number:  2
index:  5
summary:
 The patient is a 34-year-old female with a known history of migraines with aura. Possible hypertension. She presented with a severe headache associated with visual disturbances and nausea. The headache was of sudden onset and had been persistent for the past 12 hours. Her family history includes a maternal aunt with a history of migraines.

The patient was treated with intravenous fluids, antiemetics, and a dose of sumatriptan. Symptoms improved significantly within 4 hours, and she was discharged with oral medications to manage future episodes. She was instructed to avoid known migraine triggers, such as stress and bright lights.

The patient was advised to follow up with her neurologist in one month.


In [24]:
stats(dsum['hypertension-status'],htn_6)

accuracy = 0.9
PPV = 0.75
Sensitivity = 1.0
NPV = 1.0
Specificity = 0.86


## Uncertainty

In [25]:
def get_htn_7(dsum):
    la = dsum.str.contains(r"(?i)\b(?:hypertension|htn|hypertensive)\b",regex=True)
    ln = dsum.str.contains(r"(?i)\bno\b.*\b(?:hypertension|htn|hypertensive)\b",regex=True)
    lu = dsum.str.contains(r"(?i)\bpossible\b.*\b(?:hypertension|htn|hypertensive)\b",regex=True)
    return la & ~ln & ~lu

htn_7 = get_htn_7(dsum['summary'])
fp(dsum['hypertension-status'],htn_7)

number:  1
index:  18
summary:
 The patient is a 24-year-old male with no significant past medical history. He presented with acute onset right lower quadrant abdominal pain, nausea, and fever. He has no known history of gastrointestinal disorders, surgeries, or chronic illness, and is a non-smoker who consumes alcohol occasionally. Family history is non-contributory, with no known family history of abdominal disorders.

However, the patient’s maternal grandmother has a history of hypertension and diabetes, which was noted during family history assessment.

The prognosis is excellent post-appendectomy, as the condition was identified early, and there were no signs of rupture or peritonitis. Barring complications, the patient is expected to return to full function within 2-4 weeks.

The patient underwent a laparoscopic appendectomy within 12 hours of presentation. Intraoperative findings were consistent with uncomplicated appendicitis. Post-operative recovery was uneventful, and he tole

In [26]:
stats(dsum['hypertension-status'],htn_7)

accuracy = 0.95
PPV = 0.86
Sensitivity = 1.0
NPV = 1.0
Specificity = 0.93


## Extracting Values (Age)
- really only works this well due to regularities in ChatGPT output used to initialize the synthetic discharge summaries.

In [27]:
ages = dsum['summary'].str.extract(r"(\d{1,3})-year-old")
ages.head()

,0
0,67
1,49
2,62
3,55
4,58


In [28]:
pd.concat([ages,dsum['Age']],axis=1)

,0,Age
0,67,67
1,49,49
2,62,62
3,55,55
4,58,58
5,34,34
6,58,58
7,26,26
8,63,63
9,58,58


## LLM
https://huggingface.co/Qwen/Qwen3-0.6B

In [29]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


/home/elmo/miniconda3/envs/leaderboard/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


thinking content: <think>
Okay, the user wants a short introduction to a large language model. Let me start by recalling what I know about them. A large language model is a type of artificial intelligence that can understand and generate human language. They're trained on massive datasets to learn patterns and improve their responses.

I should mention their ability to understand and generate text, maybe some examples. Also, their applications in various fields like customer service, content creation, etc. Need to keep it concise but informative. Avoid technical jargon if possible. Make sure it's a friendly and engaging opening. Let me check if I'm covering all key points without getting too detailed. Yes, that should work.
</think>
content: A large language model (LLM) is an artificial intelligence system designed to understand and generate human language. It learns from vast datasets, enabling it to comprehend context, understand nuances, and produce coherent responses. These models 

In [30]:
def llm(prompt):
    
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=2000 #shrinking this to truncate output
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 
    
    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0
    
    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    
    print("\n\nthinking content:\n", thinking_content)
    print("\n\ncontent:\n", content)

In [31]:
ds = dsum['summary'].iloc[0]

prompt = f""" 
You're performing medical chart review to determine if the patient has hypertension:

If hypertension is present print #Y# and if not present print #N#

Chart:
{ds}
"""

print("Original Discharge Summary:\n", ds)

llm(prompt)

Original Discharge Summary:
 The patient is a 67-year-old female with a history of hypertension and type 2 diabetes mellitus. She presented with a 3-day history of fever, productive cough, and shortness of breath. Her physical exam revealed crackles in the right lower lung field. Family history includes a mother with a history of lung cancer and a father who had hypertension and coronary artery disease.

The patient was diagnosed with community-acquired pneumonia (CAP) and treated with a course of intravenous ceftriaxone and azithromycin. Her fever resolved by day three, and she demonstrated significant clinical improvement. The patient was switched to oral antibiotics on day five and discharged after a 7-day course.

She was advised to monitor her blood glucose levels closely and avoid smoking, as it may worsen her lung condition. The patient was instructed to follow up with her primary care physician within one week to ensure continued improvement.


thinking content:
 <think>
Okay, 

### Let's revisit the family history

In [32]:
fp(dsum['hypertension-status'],htn_7)

number:  1
index:  18
summary:
 The patient is a 24-year-old male with no significant past medical history. He presented with acute onset right lower quadrant abdominal pain, nausea, and fever. He has no known history of gastrointestinal disorders, surgeries, or chronic illness, and is a non-smoker who consumes alcohol occasionally. Family history is non-contributory, with no known family history of abdominal disorders.

However, the patient’s maternal grandmother has a history of hypertension and diabetes, which was noted during family history assessment.

The prognosis is excellent post-appendectomy, as the condition was identified early, and there were no signs of rupture or peritonitis. Barring complications, the patient is expected to return to full function within 2-4 weeks.

The patient underwent a laparoscopic appendectomy within 12 hours of presentation. Intraoperative findings were consistent with uncomplicated appendicitis. Post-operative recovery was uneventful, and he tole

In [33]:
ds = dsum['summary'].loc[18]

prompt = f""" 
You're performing medical chart review to determine if the patient has hypertension:

If hypertension is present print #Y# and if not present print #N#

Chart:
{ds}
"""

print("Original Discharge Summary:", ds)

llm(prompt)

Original Discharge Summary: The patient is a 24-year-old male with no significant past medical history. He presented with acute onset right lower quadrant abdominal pain, nausea, and fever. He has no known history of gastrointestinal disorders, surgeries, or chronic illness, and is a non-smoker who consumes alcohol occasionally. Family history is non-contributory, with no known family history of abdominal disorders.

However, the patient’s maternal grandmother has a history of hypertension and diabetes, which was noted during family history assessment.

The prognosis is excellent post-appendectomy, as the condition was identified early, and there were no signs of rupture or peritonitis. Barring complications, the patient is expected to return to full function within 2-4 weeks.

The patient underwent a laparoscopic appendectomy within 12 hours of presentation. Intraoperative findings were consistent with uncomplicated appendicitis. Post-operative recovery was uneventful, and he tolerate